In [1]:
import sys
sys.path.append('/home/rd804/particle_transformer')
import networks.example_ParticleTransformer as part
import networks.example_ParticleTransformer_finetune as part_finetune
from weaver.utils.dataset import DataConfig
import torch
from weaver.nn.model.ParticleTransformer import ParticleTransformer
from weaver.utils.logger import _logger

In [2]:
cd ..

/home/rd804/particle_transformer


In [3]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import h5py
import numpy as np 

In [6]:
data_config_file = 'data/JetClass/JetClass_kin.yaml'

In [7]:
data_config = DataConfig.load(data_config_file, load_observers=False, load_reweight_info=False)

In [8]:
part_features = torch.from_numpy(np.load('data/part_features.npy'))
part_4momenta = np.load('data/part_4momenta.npy')
part_labels = torch.from_numpy(np.load('data/part_labels.npy'))

mask = torch.tensor((part_4momenta!=[0,0,0,0])[...,0]).unsqueeze(-1)

part_4momenta = torch.tensor(part_4momenta).to(device)
vector = torch.swapaxes(part_4momenta, 1, 2).to(device)
features = torch.swapaxes(part_features, 1, 2).to(device)
mask = torch.swapaxes(mask, 1, 2).to(device)

In [9]:
# count number of parameters

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [10]:
model, model_info = part.get_model(data_config)
model.to(device)
#model.mod.requires_grad_(False)
print(f'The model has {count_parameters(model):,} trainable parameters')

path_part = 'models/ParT_kin.pt'
model.load_state_dict(torch.load(path_part))

input_dim: 7
The model has 2,142,166 trainable parameters


<All keys matched successfully>

In [9]:
mini_batch = 2000
pred= []


for i in range(len(features)//mini_batch):
    with torch.no_grad():
        pred.append(model.forward(features[i*mini_batch:(i+1)*mini_batch], vector[i*mini_batch:(i+1)*mini_batch], mask[i*mini_batch:(i+1)*mini_batch]))
    print(f'Batch {i} done')



#with torch.no_grad():
 #   pred = model.forward(features[:100], vector[:100], mask[:100])


/home/rd804/.conda/envs/part/lib/python3.10/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Batch 0 done
Batch 1 done
Batch 2 done
Batch 3 done
Batch 4 done
Batch 5 done
Batch 6 done
Batch 7 done
Batch 8 done
Batch 9 done
Batch 10 done
Batch 11 done
Batch 12 done
Batch 13 done
Batch 14 done
Batch 15 done
Batch 16 done
Batch 17 done
Batch 18 done
Batch 19 done
Batch 20 done
Batch 21 done
Batch 22 done
Batch 23 done
Batch 24 done
Batch 25 done
Batch 26 done
Batch 27 done
Batch 28 done
Batch 29 done
Batch 30 done
Batch 31 done
Batch 32 done
Batch 33 done
Batch 34 done
Batch 35 done
Batch 36 done
Batch 37 done
Batch 38 done
Batch 39 done
Batch 40 done
Batch 41 done
Batch 42 done
Batch 43 done
Batch 44 done
Batch 45 done
Batch 46 done
Batch 47 done
Batch 48 done
Batch 49 done
Batch 50 done
Batch 51 done
Batch 52 done
Batch 53 done
Batch 54 done
Batch 55 done
Batch 56 done
Batch 57 done
Batch 58 done
Batch 59 done
Batch 60 done
Batch 61 done
Batch 62 done
Batch 63 done
Batch 64 done
Batch 65 done
Batch 66 done
Batch 67 done
Batch 68 done
Batch 69 done
Batch 70 done
Batch 71 done
Ba

In [10]:
ypred = np.concatenate([p.cpu().numpy() for p in pred], axis=0)

In [11]:
ypred.shape

(404000, 2)

In [12]:
np.save('data/pred.npy', ypred)

In [14]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

def R30(y_true,y_pred,thr): 
    #to find the threshold that gives TPR of 50%, and the false positive rate at a TPR of 50% 
    fpr_, tpr_, thresholds_ = roc_curve(y_true, y_pred)

    t50 = thresholds_[np.argmin(np.absolute(tpr_-thr))]

    fpr50 = fpr_[thresholds_==t50].item()
    tpr50 = tpr_[thresholds_==t50].item()
    if fpr50>0:
        print('R30 at tpr ', tpr50, ' is ',1/fpr50)
    
    #print(tpr50)
    return  t50, fpr50, tpr50
    

In [18]:
R30(part_labels[:,1], ypred[:,1], 0.3)

R30 at tpr  0.3000089146864507  is  4.490101539760482


(-0.08696413, 0.22271211266490504, 0.3000089146864507)